step-1 :  Install PySpark

In [2]:
pip install pyspark


step-2 : Import required libraries

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count, desc, col


Step 3: Initialize Spark Session


In [4]:
spark = SparkSession.builder \
    .appName("Big Data Analysis - NYC Taxi") \
    .getOrCreate()


Step 4: Downloading the Dataset

In [10]:
# Download January 2023 NYC Yellow Taxi Trip Data (Parquet format)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O yellow_tripdata_2023-01.parquet

--2025-06-22 06:01:28--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.50, 18.160.201.126, 18.160.201.5, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   220MB/s    in 0.2s    

2025-06-22 06:01:28 (220 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



 Step 5: Load the large dataset


In [12]:
df = spark.read.parquet("yellow_tripdata_2023-01.parquet")

# Show sample data
df.show(5)
df.printSchema()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|       1.0|                 N|         161|         141|           2|        9.3|  1.0|    0.5|       0.

 Step 6: Basic data cleaning

In [13]:
df_clean = df.dropna(subset=["trip_distance", "passenger_count", "fare_amount"])

# Filter out invalid records
df_clean = df_clean.filter((col("trip_distance") > 0) & (col("fare_amount") > 0))

Step 7: Exploratory Data Analysis

In [14]:
total_trips = df_clean.count()
print("Total Trips:", total_trips)

# Average trip distance
df_clean.select(avg("trip_distance").alias("Avg Trip Distance")).show()

# Most common payment types
df_clean.groupBy("payment_type") \
        .agg(count("*").alias("count")) \
        .orderBy(desc("count")) \
        .show()

# Average fare per passenger count
df_clean.groupBy("passenger_count") \
        .agg(avg("fare_amount").alias("Avg Fare")) \
        .orderBy("passenger_count") \
        .show()

Total Trips: 2934181
+------------------+
| Avg Trip Distance|
+------------------+
|3.4617694716173912|
+------------------+

+------------+-------+
|payment_type|  count|
+------------+-------+
|           1|2390461|
|           2| 517040|
|           4|  16535|
|           3|  10145|
+------------+-------+

+---------------+------------------+
|passenger_count|          Avg Fare|
+---------------+------------------+
|            0.0|16.168202510359798|
|            1.0|18.074408467255598|
|            2.0| 20.40332980243411|
|            3.0| 19.86253246753282|
|            4.0|20.882765446795513|
|            5.0| 17.96817410966637|
|            6.0|18.043596657262192|
|            7.0|              67.8|
|            8.0| 82.49571428571429|
+---------------+------------------+



Step 8: Deriving Insights

In [15]:
df_clean.groupBy("payment_type") \
        .agg(avg("trip_distance").alias("Avg Trip Distance")) \
        .orderBy(desc("Avg Trip Distance")) \
        .show()

# Total revenue generated (approx.)
total_revenue = df_clean.agg({"fare_amount": "sum"}).collect()[0][0]
print(f"Total Revenue Generated: ${total_revenue:,.2f}")


+------------+------------------+
|payment_type| Avg Trip Distance|
+------------+------------------+
|           1|3.4650946323742486|
|           2| 3.461348116200059|
|           4|3.2594647716964125|
|           3|3.0294677180877136|
+------------+------------------+

Total Revenue Generated: $54,300,005.24


Step 9: Stop Spark Session

In [16]:
spark.stop()